In [1]:
# This is a notebook to summarize all different models.
# THis notebook was not run


## Heavy Decoder
# encoder_dense_layers_trial = [[10, 8], [12, 10], [14, 12], [16, 14], [18, 16], [20, 18],[22, 20]]
# decoder_dense_layers_trial = [[6, 8, 10, 12], [8, 10, 12, 14], [10, 12, 14, 16], [12, 14, 16, 18], [14, 16, 18, 20], 
#                               [16, 18, 20, 22], [18, 20, 22, 24]]
# bottle_neck_trial = [8, 10, 12, 14, 16, 18]

In [2]:
#Disable Warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

In [3]:
FILE1 = 'SB-001_normalized_AM.csv'
FILE2 = 'SB-001_normalized_PM.csv'
NO_OF_COLUMNS = 24

In [4]:
#pip install tf2onnx
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
plt.ioff()
import cv2
import IPython
from tqdm import tqdm
import time
import os
import pickle
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', message=r'.*Consider either turning off auto-sharding.*')

In [5]:
run_summary = []

In [6]:
def generate_autoencoder_reports(encoder_dense_layers, bottle_neck, decoder_dense_layers):

    print(encoder_dense_layers, bottle_neck, decoder_dense_layers)
    # Load Data
    df_am = pd.read_csv(FILE1)
    df_pm = pd.read_csv(FILE2)
    combined_df = pd.concat([df_am, df_pm])

    columns_needed = ['y_am_pef', 'tempin', 'humidin', 'pm25in', 'co2in', 'tempdiffin', 'humidiffin', 'pm25diffin',
                      'pm10', 'pm25', 'o3', 'no2', 'co', 'so2', 'temp', 'windsd', 'humid', 'varp', 'dewpt', 'airp',
                      'seap', 'solrhr', 'solramnt', 'grdt', 'class']
    combined_df = combined_df.filter(columns_needed)
    df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Drop NaN
    df = df.dropna()

    # Minotity Split
    class_counts = df['class'].value_counts()
    minority_class = df[df['class'] == 0]
    majority_class = df[df['class'] == 1]

    # Removing class column from minority before augmentation
    X = minority_class.drop('class', axis=1)
    y = minority_class['class']

    # Saving minority as X_train
    X_train = X

    # Define the Input shape
    INPUT_SHAPE = X_train.shape[1]
    FILE_NAME = f"L{INPUT_SHAPE}_E{'_'.join(map(str, encoder_dense_layers))}_B{bottle_neck}_D{'_'.join(map(str, decoder_dense_layers))}"
    print("\n" + FILE_NAME + "\n")
    
    def build_autoencoder(input_shape, **kwargs):

        # Autoencoder model construction cod

      """
      Build an autoencoder model with the given configuration.

      Args:
          input_shape (int): The shape of the input data.
          **kwargs: Additional keyword arguments for configuring the autoencoder.

      Keyword Args:
          encoder_dense_layers (list): List of integers specifying the number of units for each dense layer in the encoder.
                                      Default: []
          bottle_neck (int): The number of units in the bottleneck layer. Default: input_shape // 2
          decoder_dense_layers (list): List of integers specifying the number of units for each dense layer in the decoder.
                                      Default: []
          decoder_activation (str): The activation function for the decoder output layer. Default: 'sigmoid'

      Returns:
          tuple: A tuple containing the autoencoder, encoder, and decoder models.

      Example:
          INPUT_SHAPE = 27
          encoder_dense_layers = [20, 18]  # Specify the number of units for each dense layer in the encoder
          bottle_neck = 16
          decoder_dense_layers = [18, 20]  # Specify the number of units for each dense layer in the decoder

          autoencoder, encoder, decoder = build_autoencoder(INPUT_SHAPE, encoder_dense_layers=encoder_dense_layers,
                                                            bottle_neck=bottle_neck, decoder_dense_layers=decoder_dense_layers)
          encoder.summary()
          decoder.summary()
          autoencoder.summary()
      """

      # Default parameter values
      encoder_dense_layers = kwargs.get('encoder_dense_layers', [])
      bottle_neck = kwargs.get('bottle_neck', input_shape // 2)
      decoder_dense_layers = kwargs.get('decoder_dense_layers', [])
      decoder_activation = kwargs.get('decoder_activation', 'sigmoid')

      # Autoencoder Model
      encoder_input = keras.Input(shape=(input_shape,), name="encoder")
      x = keras.layers.Flatten()(encoder_input)

      # Encoder Dense Layers
      for units in encoder_dense_layers:
          x = keras.layers.Dense(units, activation="relu")(x)

      encoder_output = keras.layers.Dense(bottle_neck, activation="relu")(x)
      encoder = keras.Model(encoder_input, encoder_output, name="encoder")

      # Decoder Model
      decoder_input = keras.Input(shape=(bottle_neck,), name="decoder")
      x = decoder_input

      # Decoder Dense Layers
      for units in decoder_dense_layers:
          x = keras.layers.Dense(units, activation="relu")(x)

      decoder_output = keras.layers.Dense(input_shape, activation=decoder_activation)(x)
      decoder = keras.Model(decoder_input, decoder_output, name="decoder")

      # Autoencoder Model
      autoencoder_input = keras.Input(shape=(input_shape,), name="input")
      encoded = encoder(autoencoder_input)
      decoded = decoder(encoded)
      autoencoder = keras.Model(autoencoder_input, decoded, name="autoencoder")

      return autoencoder, encoder, decoder

    autoencoder, encoder, decoder = build_autoencoder(INPUT_SHAPE, encoder_dense_layers=encoder_dense_layers,
                                                      bottle_neck=bottle_neck,
                                                      decoder_dense_layers=decoder_dense_layers)
    opt = keras.optimizers.Adam(learning_rate=0.001)
    autoencoder.compile(opt, loss="mse")

    print("Training model:", FILE_NAME)
    history = autoencoder.fit(X_train, X_train, epochs=200, batch_size=16, validation_split=0.25, verbose=0)
    print("Training Complete:")
    
    # Extract the loss values
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Print the last epoch's loss values
    last_epoch_loss = loss[-1]
    last_epoch_val_loss = val_loss[-1]
    print("Last epoch loss:", last_epoch_loss)
    print("Last epoch validation loss:", last_epoch_val_loss)

    # Saving history
    with open(FILE_NAME + '_history.pickle', 'wb') as file:
        pickle.dump(history.history, file)

    # Visualize losses *(MSE)
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(FILE_NAME + 'Loss.png')
    #plt.show()

    # Generate synthetic data
    num_samples = len(X_train)
    input_data = np.random.normal(size=(num_samples, INPUT_SHAPE))
    generated_data = autoencoder.predict(input_data)
    reshaped_data = generated_data.reshape(num_samples, -1)
    df_generated = pd.DataFrame(reshaped_data, columns=X_train.columns)

    # Plot correlation matrices
    corr_matrix1 = X_train.corr()
    corr_matrix2 = df_generated.corr()

    fig, axes = plt.subplots(1, 2, figsize=(18, 10))
    sns.heatmap(corr_matrix1, annot=True, cmap='coolwarm', ax=axes[0])
    axes[0].set_title('Heatmap 1')

    sns.heatmap(corr_matrix2, annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Heatmap 2')

    plt.tight_layout()
    plt.savefig(FILE_NAME + 'HeatMaps')
    #plt.show()

    # Calculate mean and standard deviation of original and synthetic datasets
    common_columns = set(X_train.columns) & set(df_generated.columns)
    results = {}

    for column in common_columns:
        mean_df1 = X_train[column].mean()
        std_df1 = X_train[column].std()
        mean_df2 = df_generated[column].mean()
        std_df2 = df_generated[column].std()

        results[column] = {'Mean_df1': mean_df1, 'Std_df1': std_df1,
                           'Mean_df2': mean_df2, 'Std_df2': std_df2}

    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv(FILE_NAME + 'mean_std.csv', index=True)

    # Plot scatter plots
    new_index = pd.RangeIndex(start=0, stop=57, step=1)
    X_train = X_train.set_index(new_index)
    common_columns = set(X_train.columns) & set(df_generated.columns)

    for column in common_columns:
        plt.scatter(X_train.index, X_train[column], color='red', label='X_train')
        plt.scatter(df_generated.index, df_generated[column], color='blue', label='df_generated')

        plt.title(f"Scatter Plot: {column}")
        plt.xlabel("Index")
        plt.ylabel(column)

        plt.legend()
        plt.savefig(FILE_NAME + "_" + column)
        #plt.show()


    # Add back the class label
    X_train['class'] = 0.0
    df_generated['class'] = 0.0
    X_train.to_csv(FILE_NAME + 'Original_minority_data.csv', index=False)
    df_generated.to_csv(FILE_NAME + 'Synthetic_minority_data.csv', index=False)

    # Generate quality report and display it
    real_data = FILE_NAME + 'Original_minority_data.csv'
    synth_data = FILE_NAME + 'Synthetic_minority_data.csv'

    # report = QualityReport(data_source=synth_data, ref_data=real_data)
    # report.run()

    # print(report.peek())
    #run_summary.append([FILE_NAME, report.peek()['raw_score'], report.peek()['grade']])
    run_summary.append([FILE_NAME, last_epoch_loss, last_epoch_val_loss])

    print("Reports Saved")
    #IPython.display.HTML(report.as_html, metadata=dict(isolated=True))
    return None

In [7]:
# Example usage
def runner(encoder_dense_layers, bottle_neck, decoder_dense_layers):
  #encoder_dense_layers = [10, 8]
  #bottle_neck = 5
  #decoder_dense_layers = [8, 10]
  #FILE_NAME = f"24_{'_'.join(map(str, encoder_dense_layers))}_{bottle_neck}_"
  FILE_NAME = f"L{NO_OF_COLUMNS}_E{'_'.join(map(str, encoder_dense_layers))}_B{bottle_neck}_D{'_'.join(map(str, decoder_dense_layers))}"   
  report = generate_autoencoder_reports(encoder_dense_layers, bottle_neck, decoder_dense_layers)
  #print(type(report))
  # Save the report object to a file
  # with open(FILE_NAME + 'quality_report.pickle', 'wb') as file:
  #     pickle.dump(report, file)

  #IPython.display.HTML(report.as_html, metadata=dict(isolated=True))

In [8]:
# here goes nothin

In [9]:
encoder_dense_layers_trial = [[10, 8], [12, 10], [14, 12], [16, 14], [18, 16], [20, 18],[22, 20]]
decoder_dense_layers_trial = [[6, 8, 10, 12], [8, 10, 12, 14], [10, 12, 14, 16], [12, 14, 16, 18], [14, 16, 18, 20], 
                              [16, 18, 20, 22], [18, 20, 22, 24]]
bottle_neck_trial = [8, 10, 12, 14, 16, 18]

In [10]:
total_iterations = len(bottle_neck_trial) * len(encoder_dense_layers_trial) * len(decoder_dense_layers_trial)
print("Total Model in Pipeline:",total_iterations)

# Print total models
for bn in bottle_neck_trial:
  for enc_layers in encoder_dense_layers_trial:
      for dec_layers in decoder_dense_layers_trial:
          print(enc_layers, bn, dec_layers)

Total Model in Pipeline: 294
[10, 8] 8 [6, 8, 10, 12]
[10, 8] 8 [8, 10, 12, 14]
[10, 8] 8 [10, 12, 14, 16]
[10, 8] 8 [12, 14, 16, 18]
[10, 8] 8 [14, 16, 18, 20]
[10, 8] 8 [16, 18, 20, 22]
[10, 8] 8 [18, 20, 22, 24]
[12, 10] 8 [6, 8, 10, 12]
[12, 10] 8 [8, 10, 12, 14]
[12, 10] 8 [10, 12, 14, 16]
[12, 10] 8 [12, 14, 16, 18]
[12, 10] 8 [14, 16, 18, 20]
[12, 10] 8 [16, 18, 20, 22]
[12, 10] 8 [18, 20, 22, 24]
[14, 12] 8 [6, 8, 10, 12]
[14, 12] 8 [8, 10, 12, 14]
[14, 12] 8 [10, 12, 14, 16]
[14, 12] 8 [12, 14, 16, 18]
[14, 12] 8 [14, 16, 18, 20]
[14, 12] 8 [16, 18, 20, 22]
[14, 12] 8 [18, 20, 22, 24]
[16, 14] 8 [6, 8, 10, 12]
[16, 14] 8 [8, 10, 12, 14]
[16, 14] 8 [10, 12, 14, 16]
[16, 14] 8 [12, 14, 16, 18]
[16, 14] 8 [14, 16, 18, 20]
[16, 14] 8 [16, 18, 20, 22]
[16, 14] 8 [18, 20, 22, 24]
[18, 16] 8 [6, 8, 10, 12]
[18, 16] 8 [8, 10, 12, 14]
[18, 16] 8 [10, 12, 14, 16]
[18, 16] 8 [12, 14, 16, 18]
[18, 16] 8 [14, 16, 18, 20]
[18, 16] 8 [16, 18, 20, 22]
[18, 16] 8 [18, 20, 22, 24]
[20, 18] 8 [6

In [ ]:
'''
Type	Bottleneck	Model	Training Loss	Validation Loss
Heavy Decoder	B8	L24_E10_8_B8_D6_8_10_12	0.019926628	0.031914495
Heavy Decoder	B8	L24_E10_8_B8_D8_10_12_14	0.012820457	0.020630909
Heavy Decoder	B8	L24_E10_8_B8_D10_12_14_16	0.015678098	0.026914198
Heavy Decoder	B8	L24_E10_8_B8_D12_14_16_18	0.010084689	0.01811916
Heavy Decoder	B8	L24_E10_8_B8_D14_16_18_20	0.013961583	0.024084207
Heavy Decoder	B8	L24_E10_8_B8_D16_18_20_22	0.011970182	0.019392315
Heavy Decoder	B8	L24_E10_8_B8_D18_20_22_24	0.011579308	0.01802923
Heavy Decoder	B8	L24_E12_10_B8_D6_8_10_12	0.017853063	0.028062725
Heavy Decoder	B8	L24_E12_10_B8_D8_10_12_14	0.011113857	0.019850979
Heavy Decoder	B8	L24_E12_10_B8_D10_12_14_16	0.013588903	0.021442031
Heavy Decoder	B8	L24_E12_10_B8_D12_14_16_18	0.011265877	0.020352019
Heavy Decoder	B8	L24_E12_10_B8_D14_16_18_20	0.013636555	0.024873422
Heavy Decoder	B8	L24_E12_10_B8_D16_18_20_22	0.011745661	0.017587217
Heavy Decoder	B8	L24_E12_10_B8_D18_20_22_24	0.009548418	0.016957927
Heavy Decoder	B8	L24_E14_12_B8_D6_8_10_12	0.012102078	0.020032769
Heavy Decoder	B8	L24_E14_12_B8_D8_10_12_14	0.014465909	0.026334645
Heavy Decoder	B8	L24_E14_12_B8_D10_12_14_16	0.014891272	0.02552733
Heavy Decoder	B8	L24_E14_12_B8_D12_14_16_18	0.010671251	0.017016433
Heavy Decoder	B8	L24_E14_12_B8_D14_16_18_20	0.014918462	0.028301496
Heavy Decoder	B8	L24_E14_12_B8_D16_18_20_22	0.008329754	0.014043828
Heavy Decoder	B8	L24_E14_12_B8_D18_20_22_24	0.009826655	0.017809462
Heavy Decoder	B8	L24_E16_14_B8_D6_8_10_12	0.013053085	0.019166024
Heavy Decoder	B8	L24_E16_14_B8_D8_10_12_14	0.013206534	0.01936901
Heavy Decoder	B8	L24_E16_14_B8_D10_12_14_16	0.011198245	0.019090597
Heavy Decoder	B8	L24_E16_14_B8_D12_14_16_18	0.009617953	0.017003838
Heavy Decoder	B8	L24_E16_14_B8_D14_16_18_20	0.008466826	0.013237664
Heavy Decoder	B8	L24_E16_14_B8_D16_18_20_22	0.012426714	0.020856777
Heavy Decoder	B8	L24_E16_14_B8_D18_20_22_24	0.010172438	0.018499367
Heavy Decoder	B8	L24_E18_16_B8_D6_8_10_12	0.012117364	0.018034356
Heavy Decoder	B8	L24_E18_16_B8_D8_10_12_14	0.012578387	0.0200841
Heavy Decoder	B8	L24_E18_16_B8_D10_12_14_16	0.015688391	0.029850515
Heavy Decoder	B8	L24_E18_16_B8_D12_14_16_18	0.011237763	0.018727092
Heavy Decoder	B8	L24_E18_16_B8_D14_16_18_20	0.008583124	0.015225613
Heavy Decoder	B8	L24_E18_16_B8_D16_18_20_22	0.015706472	0.02766593
Heavy Decoder	B8	L24_E18_16_B8_D18_20_22_24	0.008851178	0.018214721
Heavy Decoder	B8	L24_E20_18_B8_D6_8_10_12	0.011311103	0.020479875
Heavy Decoder	B8	L24_E20_18_B8_D8_10_12_14	0.018034931	0.027819661
Heavy Decoder	B8	L24_E20_18_B8_D10_12_14_16	0.011792514	0.018304996
Heavy Decoder	B8	L24_E20_18_B8_D12_14_16_18	0.011361422	0.017289581
Heavy Decoder	B8	L24_E20_18_B8_D14_16_18_20	0.009899212	0.017126333
Heavy Decoder	B8	L24_E20_18_B8_D16_18_20_22	0.010891235	0.017784117
Heavy Decoder	B8	L24_E20_18_B8_D18_20_22_24	0.011735962	0.019991931
Heavy Decoder	B8	L24_E22_20_B8_D6_8_10_12	0.011352683	0.018977832
Heavy Decoder	B8	L24_E22_20_B8_D8_10_12_14	0.011352731	0.018756287
Heavy Decoder	B8	L24_E22_20_B8_D10_12_14_16	0.012082977	0.020539038
Heavy Decoder	B8	L24_E22_20_B8_D12_14_16_18	0.011525488	0.018882483
Heavy Decoder	B8	L24_E22_20_B8_D14_16_18_20	0.009868192	0.016412564
Heavy Decoder	B8	L24_E22_20_B8_D16_18_20_22	0.009056972	0.014489492
Heavy Decoder	B8	L24_E22_20_B8_D18_20_22_24	0.007857903	0.014609795
Heavy Decoder	B10	L24_E10_8_B10_D6_8_10_12	0.01617749	0.024610464
Heavy Decoder	B10	L24_E10_8_B10_D8_10_12_14	0.01199057	0.018966563
Heavy Decoder	B10	L24_E10_8_B10_D10_12_14_16	0.011335118	0.019346694
Heavy Decoder	B10	L24_E10_8_B10_D12_14_16_18	0.011492903	0.018522641
Heavy Decoder	B10	L24_E10_8_B10_D14_16_18_20	0.014547646	0.033157822
Heavy Decoder	B10	L24_E10_8_B10_D16_18_20_22	0.012943004	0.021177737
Heavy Decoder	B10	L24_E10_8_B10_D18_20_22_24	0.01279323	0.021971969
Heavy Decoder	B10	L24_E12_10_B10_D6_8_10_12	0.020811589	0.030136764
Heavy Decoder	B10	L24_E12_10_B10_D8_10_12_14	0.0120162	0.018875588
Heavy Decoder	B10	L24_E12_10_B10_D10_12_14_16	0.012202449	0.018837618
Heavy Decoder	B10	L24_E12_10_B10_D12_14_16_18	0.016870109	0.027344983
Heavy Decoder	B10	L24_E12_10_B10_D14_16_18_20	0.011805949	0.019425165
Heavy Decoder	B10	L24_E12_10_B10_D16_18_20_22	0.012574358	0.020960186
Heavy Decoder	B10	L24_E12_10_B10_D18_20_22_24	0.017579434	0.02617182
Heavy Decoder	B10	L24_E14_12_B10_D6_8_10_12	0.014620586	0.021581242
Heavy Decoder	B10	L24_E14_12_B10_D8_10_12_14	0.012811122	0.019523477
Heavy Decoder	B10	L24_E14_12_B10_D10_12_14_16	0.011655763	0.019696247
Heavy Decoder	B10	L24_E14_12_B10_D12_14_16_18	0.015587794	0.022968223
Heavy Decoder	B10	L24_E14_12_B10_D14_16_18_20	0.010810665	0.017720209
Heavy Decoder	B10	L24_E14_12_B10_D16_18_20_22	0.010457464	0.020521071
Heavy Decoder	B10	L24_E14_12_B10_D18_20_22_24	0.009871496	0.01755807
Heavy Decoder	B10	L24_E16_14_B10_D6_8_10_12	0.019933959	0.029187543
Heavy Decoder	B10	L24_E16_14_B10_D8_10_12_14	0.013325458	0.01986823
Heavy Decoder	B10	L24_E16_14_B10_D10_12_14_16	0.008458379	0.016088435
Heavy Decoder	B10	L24_E16_14_B10_D12_14_16_18	0.014076924	0.024699261
Heavy Decoder	B10	L24_E16_14_B10_D14_16_18_20	0.007200642	0.015180391
Heavy Decoder	B10	L24_E16_14_B10_D16_18_20_22	0.009967934	0.017722603
Heavy Decoder	B10	L24_E16_14_B10_D18_20_22_24	0.011696822	0.020563954
Heavy Decoder	B10	L24_E18_16_B10_D6_8_10_12	0.013405855	0.020098869
Heavy Decoder	B10	L24_E18_16_B10_D8_10_12_14	0.01308763	0.020692952
Heavy Decoder	B10	L24_E18_16_B10_D10_12_14_16	0.016480608	0.024379078
Heavy Decoder	B10	L24_E18_16_B10_D12_14_16_18	0.012067924	0.01809236
Heavy Decoder	B10	L24_E18_16_B10_D14_16_18_20	0.009437418	0.018321879
Heavy Decoder	B10	L24_E18_16_B10_D16_18_20_22	0.010487803	0.019605892
Heavy Decoder	B10	L24_E18_16_B10_D18_20_22_24	0.009581531	0.017845809
Heavy Decoder	B10	L24_E20_18_B10_D6_8_10_12	0.013624102	0.021669757
Heavy Decoder	B10	L24_E20_18_B10_D8_10_12_14	0.016905304	0.023673935
Heavy Decoder	B10	L24_E20_18_B10_D10_12_14_16	0.009542614	0.015575937
Heavy Decoder	B10	L24_E20_18_B10_D12_14_16_18	0.011156549	0.017944233
Heavy Decoder	B10	L24_E20_18_B10_D14_16_18_20	0.01248119	0.02218469
Heavy Decoder	B10	L24_E20_18_B10_D16_18_20_22	0.009186707	0.01692301
Heavy Decoder	B10	L24_E20_18_B10_D18_20_22_24	0.008503783	0.016528755
Heavy Decoder	B10	L24_E22_20_B10_D6_8_10_12	0.014513289	0.022845794
Heavy Decoder	B10	L24_E22_20_B10_D8_10_12_14	0.01136691	0.019605732
Heavy Decoder	B10	L24_E22_20_B10_D10_12_14_16	0.008032629	0.013977082
Heavy Decoder	B10	L24_E22_20_B10_D12_14_16_18	0.006152914	0.012730959
Heavy Decoder	B10	L24_E22_20_B10_D14_16_18_20	0.010961058	0.017352784
Heavy Decoder	B10	L24_E22_20_B10_D16_18_20_22	0.008299788	0.013920601
Heavy Decoder	B10	L24_E22_20_B10_D18_20_22_24	0.010940257	0.017491288
Heavy Decoder	B12	L24_E10_8_B12_D6_8_10_12	0.02085226	0.02993325
Heavy Decoder	B12	L24_E10_8_B12_D8_10_12_14	0.020580277	0.03002814
Heavy Decoder	B12	L24_E10_8_B12_D10_12_14_16	0.012200225	0.020292405
Heavy Decoder	B12	L24_E10_8_B12_D12_14_16_18	0.010822658	0.018489391
Heavy Decoder	B12	L24_E10_8_B12_D14_16_18_20	0.011028523	0.02141482
Heavy Decoder	B12	L24_E10_8_B12_D16_18_20_22	0.011975507	0.019119505
Heavy Decoder	B12	L24_E10_8_B12_D18_20_22_24	0.012728666	0.018916126
Heavy Decoder	B12	L24_E12_10_B12_D6_8_10_12	0.014082247	0.020918448
Heavy Decoder	B12	L24_E12_10_B12_D8_10_12_14	0.01307523	0.019432681
Heavy Decoder	B12	L24_E12_10_B12_D10_12_14_16	0.01389892	0.023466131
Heavy Decoder	B12	L24_E12_10_B12_D12_14_16_18	0.012360448	0.019326817
Heavy Decoder	B12	L24_E12_10_B12_D14_16_18_20	0.01406364	0.021917408
Heavy Decoder	B12	L24_E12_10_B12_D16_18_20_22	0.007908826	0.014001522
Heavy Decoder	B12	L24_E12_10_B12_D18_20_22_24	0.009199761	0.01670053
Heavy Decoder	B12	L24_E14_12_B12_D6_8_10_12	0.015532128	0.025982391
Heavy Decoder	B12	L24_E14_12_B12_D8_10_12_14	0.012011804	0.019856552
Heavy Decoder	B12	L24_E14_12_B12_D10_12_14_16	0.01212867	0.019433295
Heavy Decoder	B12	L24_E14_12_B12_D12_14_16_18	0.012212859	0.019038336
Heavy Decoder	B12	L24_E14_12_B12_D14_16_18_20	0.010627808	0.017710555
Heavy Decoder	B12	L24_E14_12_B12_D16_18_20_22	0.00912258	0.017804381
Heavy Decoder	B12	L24_E14_12_B12_D18_20_22_24	0.011081631	0.018871486
Heavy Decoder	B12	L24_E16_14_B12_D6_8_10_12	0.017017398	0.02913595
Heavy Decoder	B12	L24_E16_14_B12_D8_10_12_14	0.011995218	0.019841345
Heavy Decoder	B12	L24_E16_14_B12_D10_12_14_16	0.008422054	0.015579781
Heavy Decoder	B12	L24_E16_14_B12_D12_14_16_18	0.011621462	0.019679062
Heavy Decoder	B12	L24_E16_14_B12_D14_16_18_20	0.008911361	0.018223668
Heavy Decoder	B12	L24_E16_14_B12_D16_18_20_22	0.010081922	0.019315099
Heavy Decoder	B12	L24_E16_14_B12_D18_20_22_24	0.008505468	0.015559361
Heavy Decoder	B12	L24_E18_16_B12_D6_8_10_12	0.01224714	0.018210115
Heavy Decoder	B12	L24_E18_16_B12_D8_10_12_14	0.014781563	0.02319755
Heavy Decoder	B12	L24_E18_16_B12_D10_12_14_16	0.009717763	0.018853044
Heavy Decoder	B12	L24_E18_16_B12_D12_14_16_18	0.013606529	0.019454964
Heavy Decoder	B12	L24_E18_16_B12_D14_16_18_20	0.014664819	0.023023386
Heavy Decoder	B12	L24_E18_16_B12_D16_18_20_22	0.007571518	0.014123177
Heavy Decoder	B12	L24_E18_16_B12_D18_20_22_24	0.009623448	0.016357137
Heavy Decoder	B12	L24_E20_18_B12_D6_8_10_12	0.012537383	0.019836985
Heavy Decoder	B12	L24_E20_18_B12_D8_10_12_14	0.012002917	0.018898582
Heavy Decoder	B12	L24_E20_18_B12_D10_12_14_16	0.015654644	0.027118146
Heavy Decoder	B12	L24_E20_18_B12_D12_14_16_18	0.009979123	0.015078695
Heavy Decoder	B12	L24_E20_18_B12_D14_16_18_20	0.011016244	0.01805836
Heavy Decoder	B12	L24_E20_18_B12_D16_18_20_22	0.007649989	0.013232651
Heavy Decoder	B12	L24_E20_18_B12_D18_20_22_24	0.010399478	0.017442474
Heavy Decoder	B12	L24_E22_20_B12_D6_8_10_12	0.017334213	0.027128486
Heavy Decoder	B12	L24_E22_20_B12_D8_10_12_14	0.013127338	0.020654367
Heavy Decoder	B12	L24_E22_20_B12_D10_12_14_16	0.008563383	0.013469688
Heavy Decoder	B12	L24_E22_20_B12_D12_14_16_18	0.011292834	0.018262001
Heavy Decoder	B12	L24_E22_20_B12_D14_16_18_20	0.011746749	0.018651644
Heavy Decoder	B12	L24_E22_20_B12_D16_18_20_22	0.009583009	0.017767536
Heavy Decoder	B12	L24_E22_20_B12_D18_20_22_24	0.009587358	0.018937884
Heavy Decoder	B14	L24_E10_8_B14_D6_8_10_12	0.013399748	0.019942623
Heavy Decoder	B14	L24_E10_8_B14_D8_10_12_14	0.013856885	0.024949348
Heavy Decoder	B14	L24_E10_8_B14_D10_12_14_16	0.012476523	0.01968432
Heavy Decoder	B14	L24_E10_8_B14_D12_14_16_18	0.012295647	0.01982256
Heavy Decoder	B14	L24_E10_8_B14_D14_16_18_20	0.011350309	0.018674087
Heavy Decoder	B14	L24_E10_8_B14_D16_18_20_22	0.011908285	0.019959204
Heavy Decoder	B14	L24_E10_8_B14_D18_20_22_24	0.011623992	0.017855722
Heavy Decoder	B14	L24_E12_10_B14_D6_8_10_12	0.020620164	0.030022955
Heavy Decoder	B14	L24_E12_10_B14_D8_10_12_14	0.012618531	0.019855125
Heavy Decoder	B14	L24_E12_10_B14_D10_12_14_16	0.012234853	0.017830957
Heavy Decoder	B14	L24_E12_10_B14_D12_14_16_18	0.008691774	0.015750788
Heavy Decoder	B14	L24_E12_10_B14_D14_16_18_20	0.01528099	0.025790023
Heavy Decoder	B14	L24_E12_10_B14_D16_18_20_22	0.01659053	0.028817531
Heavy Decoder	B14	L24_E12_10_B14_D18_20_22_24	0.009748182	0.018525895
Heavy Decoder	B14	L24_E14_12_B14_D6_8_10_12	0.015544613	0.023030173
Heavy Decoder	B14	L24_E14_12_B14_D8_10_12_14	0.012619214	0.020888973
Heavy Decoder	B14	L24_E14_12_B14_D10_12_14_16	0.01612485	0.027219275
Heavy Decoder	B14	L24_E14_12_B14_D12_14_16_18	0.011722335	0.019752827
Heavy Decoder	B14	L24_E14_12_B14_D14_16_18_20	0.010903643	0.02102687
Heavy Decoder	B14	L24_E14_12_B14_D16_18_20_22	0.009975309	0.01778199
Heavy Decoder	B14	L24_E14_12_B14_D18_20_22_24	0.008371046	0.014563169
Heavy Decoder	B14	L24_E16_14_B14_D6_8_10_12	0.016914401	0.026733216
Heavy Decoder	B14	L24_E16_14_B14_D8_10_12_14	0.020243434	0.029316984
Heavy Decoder	B14	L24_E16_14_B14_D10_12_14_16	0.011177752	0.019731807
Heavy Decoder	B14	L24_E16_14_B14_D12_14_16_18	0.011791554	0.018150209
Heavy Decoder	B14	L24_E16_14_B14_D14_16_18_20	0.007874776	0.013618091
Heavy Decoder	B14	L24_E16_14_B14_D16_18_20_22	0.011639429	0.018968469
Heavy Decoder	B14	L24_E16_14_B14_D18_20_22_24	0.012033977	0.018575193
Heavy Decoder	B14	L24_E18_16_B14_D6_8_10_12	0.012369959	0.019322069
Heavy Decoder	B14	L24_E18_16_B14_D8_10_12_14	0.011988897	0.020933587
Heavy Decoder	B14	L24_E18_16_B14_D10_12_14_16	0.010859053	0.01884171
Heavy Decoder	B14	L24_E18_16_B14_D12_14_16_18	0.011526885	0.027072664
Heavy Decoder	B14	L24_E18_16_B14_D14_16_18_20	0.009165065	0.015636789
Heavy Decoder	B14	L24_E18_16_B14_D16_18_20_22	0.008879378	0.017426174
Heavy Decoder	B14	L24_E18_16_B14_D18_20_22_24	0.008176344	0.014083626
Heavy Decoder	B14	L24_E20_18_B14_D6_8_10_12	0.008795007	0.014211965
Heavy Decoder	B14	L24_E20_18_B14_D8_10_12_14	0.013111577	0.019819636
Heavy Decoder	B14	L24_E20_18_B14_D10_12_14_16	0.01173713	0.018903952
Heavy Decoder	B14	L24_E20_18_B14_D12_14_16_18	0.011669183	0.017639371
Heavy Decoder	B14	L24_E20_18_B14_D14_16_18_20	0.012137257	0.018146161
Heavy Decoder	B14	L24_E20_18_B14_D16_18_20_22	0.006699539	0.013598016
Heavy Decoder	B14	L24_E20_18_B14_D18_20_22_24	0.00774019	0.013631775
Heavy Decoder	B14	L24_E22_20_B14_D6_8_10_12	0.011925475	0.020186467
Heavy Decoder	B14	L24_E22_20_B14_D8_10_12_14	0.010796593	0.018070003
Heavy Decoder	B14	L24_E22_20_B14_D10_12_14_16	0.011827203	0.018339898
Heavy Decoder	B14	L24_E22_20_B14_D12_14_16_18	0.01117596	0.017975174
Heavy Decoder	B14	L24_E22_20_B14_D14_16_18_20	0.006781273	0.013291588
Heavy Decoder	B14	L24_E22_20_B14_D16_18_20_22	0.011013284	0.017925305
Heavy Decoder	B14	L24_E22_20_B14_D18_20_22_24	0.008249566	0.016104965
Heavy Decoder	B16	L24_E10_8_B16_D6_8_10_12	0.019243199	0.031772904
Heavy Decoder	B16	L24_E10_8_B16_D8_10_12_14	0.008971762	0.01557119
Heavy Decoder	B16	L24_E10_8_B16_D10_12_14_16	0.017910106	0.026622515
Heavy Decoder	B16	L24_E10_8_B16_D12_14_16_18	0.017932832	0.027617322
Heavy Decoder	B16	L24_E10_8_B16_D14_16_18_20	0.015073754	0.023402266
Heavy Decoder	B16	L24_E10_8_B16_D16_18_20_22	0.01253855	0.020166595
Heavy Decoder	B16	L24_E10_8_B16_D18_20_22_24	0.011598252	0.02000675
Heavy Decoder	B16	L24_E12_10_B16_D6_8_10_12	0.016252495	0.023979126
Heavy Decoder	B16	L24_E12_10_B16_D8_10_12_14	0.011826775	0.019730644
Heavy Decoder	B16	L24_E12_10_B16_D10_12_14_16	0.010826796	0.018572234
Heavy Decoder	B16	L24_E12_10_B16_D12_14_16_18	0.011876215	0.019075425
Heavy Decoder	B16	L24_E12_10_B16_D14_16_18_20	0.010589904	0.020410037
Heavy Decoder	B16	L24_E12_10_B16_D16_18_20_22	0.010889174	0.018488754
Heavy Decoder	B16	L24_E12_10_B16_D18_20_22_24	0.006699296	0.013336871
Heavy Decoder	B16	L24_E14_12_B16_D6_8_10_12	0.009549412	0.015883701
Heavy Decoder	B16	L24_E14_12_B16_D8_10_12_14	0.018290259	0.028182514
Heavy Decoder	B16	L24_E14_12_B16_D10_12_14_16	0.013240386	0.030265464
Heavy Decoder	B16	L24_E14_12_B16_D12_14_16_18	0.01175663	0.018449938
Heavy Decoder	B16	L24_E14_12_B16_D14_16_18_20	0.008869783	0.015821282
Heavy Decoder	B16	L24_E14_12_B16_D16_18_20_22	0.011068586	0.018506834
Heavy Decoder	B16	L24_E14_12_B16_D18_20_22_24	0.010057555	0.0169142
Heavy Decoder	B16	L24_E16_14_B16_D6_8_10_12	0.014730976	0.02505178
Heavy Decoder	B16	L24_E16_14_B16_D8_10_12_14	0.011346896	0.023730354
Heavy Decoder	B16	L24_E16_14_B16_D10_12_14_16	0.017954346	0.029199997
Heavy Decoder	B16	L24_E16_14_B16_D12_14_16_18	0.008034728	0.014146945
Heavy Decoder	B16	L24_E16_14_B16_D14_16_18_20	0.008083167	0.016773816
Heavy Decoder	B16	L24_E16_14_B16_D16_18_20_22	0.008938136	0.018586945
Heavy Decoder	B16	L24_E16_14_B16_D18_20_22_24	0.010605593	0.017534126
Heavy Decoder	B16	L24_E18_16_B16_D6_8_10_12	0.010811215	0.016296335
Heavy Decoder	B16	L24_E18_16_B16_D8_10_12_14	0.011759428	0.019265169
Heavy Decoder	B16	L24_E18_16_B16_D10_12_14_16	0.008685778	0.016427549
Heavy Decoder	B16	L24_E18_16_B16_D12_14_16_18	0.010964803	0.0198633
Heavy Decoder	B16	L24_E18_16_B16_D14_16_18_20	0.00823427	0.014127891
Heavy Decoder	B16	L24_E18_16_B16_D16_18_20_22	0.007063529	0.015473473
Heavy Decoder	B16	L24_E18_16_B16_D18_20_22_24	0.006971302	0.013990013
Heavy Decoder	B16	L24_E20_18_B16_D6_8_10_12	0.013803542	0.020538185
Heavy Decoder	B16	L24_E20_18_B16_D8_10_12_14	0.009578071	0.018715315
Heavy Decoder	B16	L24_E20_18_B16_D10_12_14_16	0.01197136	0.019383006
Heavy Decoder	B16	L24_E20_18_B16_D12_14_16_18	0.008608559	0.015149264
Heavy Decoder	B16	L24_E20_18_B16_D14_16_18_20	0.007759089	0.013330377
Heavy Decoder	B16	L24_E20_18_B16_D16_18_20_22	0.007215609	0.013727581
Heavy Decoder	B16	L24_E20_18_B16_D18_20_22_24	0.009972167	0.018072648
Heavy Decoder	B16	L24_E22_20_B16_D6_8_10_12	0.015576638	0.02483651
Heavy Decoder	B16	L24_E22_20_B16_D8_10_12_14	0.011603369	0.018872503
Heavy Decoder	B16	L24_E22_20_B16_D10_12_14_16	0.011327572	0.018381715
Heavy Decoder	B16	L24_E22_20_B16_D12_14_16_18	0.007796038	0.014864609
Heavy Decoder	B16	L24_E22_20_B16_D14_16_18_20	0.010966417	0.022019885
Heavy Decoder	B16	L24_E22_20_B16_D16_18_20_22	0.010111888	0.017296817
Heavy Decoder	B16	L24_E22_20_B16_D18_20_22_24	0.006754107	0.014231541
Heavy Decoder	B18	L24_E10_8_B18_D6_8_10_12	0.018976871	0.029170567
Heavy Decoder	B18	L24_E10_8_B18_D8_10_12_14	0.014238815	0.022090834
Heavy Decoder	B18	L24_E10_8_B18_D10_12_14_16	0.012742657	0.021035785
Heavy Decoder	B18	L24_E10_8_B18_D12_14_16_18	0.013809389	0.021798797
Heavy Decoder	B18	L24_E10_8_B18_D14_16_18_20	0.017042201	0.026084993
Heavy Decoder	B18	L24_E10_8_B18_D16_18_20_22	0.011481716	0.018979775
Heavy Decoder	B18	L24_E10_8_B18_D18_20_22_24	0.011351201	0.018671993
Heavy Decoder	B18	L24_E12_10_B18_D6_8_10_12	0.012793745	0.019026058
Heavy Decoder	B18	L24_E12_10_B18_D8_10_12_14	0.009171666	0.014108793
Heavy Decoder	B18	L24_E12_10_B18_D10_12_14_16	0.011262167	0.018913176
Heavy Decoder	B18	L24_E12_10_B18_D12_14_16_18	0.014338922	0.02379662
Heavy Decoder	B18	L24_E12_10_B18_D14_16_18_20	0.010038973	0.017032314
Heavy Decoder	B18	L24_E12_10_B18_D16_18_20_22	0.009421322	0.016635112
Heavy Decoder	B18	L24_E12_10_B18_D18_20_22_24	0.011466952	0.018238211
Heavy Decoder	B18	L24_E14_12_B18_D6_8_10_12	0.013638024	0.024404278
Heavy Decoder	B18	L24_E14_12_B18_D8_10_12_14	0.011867948	0.019307017
Heavy Decoder	B18	L24_E14_12_B18_D10_12_14_16	0.012551965	0.019245127
Heavy Decoder	B18	L24_E14_12_B18_D12_14_16_18	0.011492564	0.018114243
Heavy Decoder	B18	L24_E14_12_B18_D14_16_18_20	0.011730838	0.019253414
Heavy Decoder	B18	L24_E14_12_B18_D16_18_20_22	0.011584658	0.019614082
Heavy Decoder	B18	L24_E14_12_B18_D18_20_22_24	0.007231295	0.013612901
Heavy Decoder	B18	L24_E16_14_B18_D6_8_10_12	0.020331744	0.029354354
Heavy Decoder	B18	L24_E16_14_B18_D8_10_12_14	0.015102516	0.024472747
Heavy Decoder	B18	L24_E16_14_B18_D10_12_14_16	0.009042187	0.016038889
Heavy Decoder	B18	L24_E16_14_B18_D12_14_16_18	0.012412716	0.018926779
Heavy Decoder	B18	L24_E16_14_B18_D14_16_18_20	0.011112468	0.018418532
Heavy Decoder	B18	L24_E16_14_B18_D16_18_20_22	0.00858915	0.014386066
Heavy Decoder	B18	L24_E16_14_B18_D18_20_22_24	0.009409947	0.016922887
Heavy Decoder	B18	L24_E18_16_B18_D6_8_10_12	0.019023109	0.029412845
Heavy Decoder	B18	L24_E18_16_B18_D8_10_12_14	0.019915041	0.029038353
Heavy Decoder	B18	L24_E18_16_B18_D10_12_14_16	0.012730284	0.019514812
Heavy Decoder	B18	L24_E18_16_B18_D12_14_16_18	0.011264757	0.019126305
Heavy Decoder	B18	L24_E18_16_B18_D14_16_18_20	0.007170556	0.014943301
Heavy Decoder	B18	L24_E18_16_B18_D16_18_20_22	0.007851245	0.014734412
Heavy Decoder	B18	L24_E18_16_B18_D18_20_22_24	0.00659407	0.013850044
Heavy Decoder	B18	L24_E20_18_B18_D6_8_10_12	0.017369058	0.027001059
Heavy Decoder	B18	L24_E20_18_B18_D8_10_12_14	0.013318332	0.022722023
Heavy Decoder	B18	L24_E20_18_B18_D10_12_14_16	0.007617832	0.014516997
Heavy Decoder	B18	L24_E20_18_B18_D12_14_16_18	0.008705925	0.016173838
Heavy Decoder	B18	L24_E20_18_B18_D14_16_18_20	0.008279722	0.014832592
Heavy Decoder	B18	L24_E20_18_B18_D16_18_20_22	0.007010866	0.014778253
Heavy Decoder	B18	L24_E20_18_B18_D18_20_22_24	0.006486921	0.013481108
Heavy Decoder	B18	L24_E22_20_B18_D6_8_10_12	0.013460875	0.022215158
Heavy Decoder	B18	L24_E22_20_B18_D8_10_12_14	0.011536745	0.020229192
Heavy Decoder	B18	L24_E22_20_B18_D10_12_14_16	0.009257151	0.01637112
Heavy Decoder	B18	L24_E22_20_B18_D12_14_16_18	0.010352857	0.017710311
Heavy Decoder	B18	L24_E22_20_B18_D14_16_18_20	0.006820044	0.013460219
Heavy Decoder	B18	L24_E22_20_B18_D16_18_20_22	0.007571639	0.013692248
Heavy Decoder	B18	L24_E22_20_B18_D18_20_22_24	0.008189476	0.017249566
'''